# Data Driven Dealings Development


*   EDA on Sales Data
*   RFM Clustering
*   Predicting Sales
*   Market Basket Analysis
*   Recommending Items per Customer







In [256]:
#!pip install D:\Dorosti\PythonPackages\jupyterthemes-0.20.0.tar.gz

In [257]:
##import jupyterthemes
#!jt -t monokai

# Reading in the Data

In [258]:
# To be able to use your data stored in your Google Drive you first need to mount your Google Drive so you can load and save files to it. 
#from google.colab import drive
import pandas as pd
import numpy as np
import pyodbc
#drive.mount('/content/gdrive')
#You'll need to put in a token which Google will generate for you as soon as you click on the link

In [259]:
conn=pyodbc.connect('Driver={Sql Server};'
                                   'Server=192.168.100.17\\SQL2019;'
                                   'Database=BarshicData_2;'
                                  'UID=sa;'
                                   'PWD=PAYA+master;'
                                   )
               
               
with open('C:\\Users\\m.dorosti\\Desktop\\RecommenderBinaryCF-main\\SalesDetail.txt' ,'r')as file:
                   query=file.read()
                   
                   
cursor = conn.cursor()   

import pandas as pd
GF=pd.read_sql_query(query,conn) 
                  
               
json_=GF

In [260]:
import pandas as pd
#data = pd.read_excel('/content/gdrive/My Drive/DDDDFolder/DDDD.xlsx')
data=GF
data.head()

,Dt_Effect,IdGds,DscGds,SalesValue,SalesAmount,IdPrsClient
0,2021-08-05,3117801,پدیلانتوس بزرگ سبز,2390000.0,1.0,21619622
1,2021-06-21,3117801,پدیلانتوس بزرگ سبز,890000.0,1.0,22560122
2,2021-08-15,3117701,پدیلانتوس متوسط سبز,890000.0,1.0,41333922
3,2021-06-21,3117701,پدیلانتوس متوسط سبز,890000.0,1.0,25652422
4,2021-06-21,3117701,پدیلانتوس متوسط سبز,600000.0,1.0,22560122


# Sparsity

In [261]:
DataPrep = data[['DscGds', 'SalesAmount', 'IdPrsClient']] #we will only use SalesItem, SalesAmount and Customer for our recommending purpose
DataPrep.head()
#DataPrep=DataPrep[0:15000]


,DscGds,SalesAmount,IdPrsClient
0,پدیلانتوس بزرگ سبز,1.0,21619622
1,پدیلانتوس بزرگ سبز,1.0,22560122
2,پدیلانتوس متوسط سبز,1.0,41333922
3,پدیلانتوس متوسط سبز,1.0,25652422
4,پدیلانتوس متوسط سبز,1.0,22560122


In [262]:
#DataPrep.head(50)

In [263]:
#SalesDate	SalesValue	SalesAmount	Customer	SalesTransactionID	SalesItem

In [264]:
DataPrep.rename(columns={'DscGds':'SalesItem','IdPrsClient':'Customer'},inplace=True)

c:\Users\m.dorosti\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [265]:
DataPrep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SalesItem    2558 non-null   object 
 1   SalesAmount  2558 non-null   float64
 2   Customer     2558 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 60.1+ KB


In [266]:
DataGrouped = DataPrep.groupby(['Customer', 'SalesItem']).sum().reset_index() # Group together
DataGrouped.head()



,Customer,SalesItem,SalesAmount
0,21604822,زاموفیلیا سبز متوسط,1.0
1,21604822,زاموفیلیا سبز کوچک,1.0
2,21604822,گلدان سفالی 50 تا 99,1.0
3,21605122,یوکا تنه دار ابلق بزرگ,1.0
4,21605922,برگ انجیری درختی متوسط,1.0


In [267]:
#len(DataGrouped.Customer.unique())

In [268]:
#make sure that no values <=0 exist
DataGroupedZero = DataGrouped.query('SalesAmount <= 0')
DataGroupedZero.head()

,Customer,SalesItem,SalesAmount


In [269]:
#in our above check we have made sure that no datarows <=0 exists. That is fine!
#only use this in case your data includes values <=0
# DataGrouped.SalesAmount.loc[DataGrouped.SalesAmount == 0] = 1 # Replace a sum of zero purchases with a one to
# DataGrouped.head()

#another interesting way to achieve the same is to use query function
#DataGrouped = DataGrouped.query('SalesAmount > 0') # Only get customers where purchase totals were positive
#DataGrouped.head()


In [270]:
import numpy as np
customers = list(np.sort(DataGrouped.Customer.unique())) # why 36 unique customers in a list and not 35? Index starts at 0!
products = list(DataGrouped.SalesItem.unique()) # Get our unique 3725 unique products that were purchased
quantity = list(DataGrouped.SalesAmount) # All of our purchases
#list function is a list of values. So customers now stores 36 unique customers.

In [271]:
from pandas import DataFrame
DfCustomerUnique = DataFrame(customers,columns=['Customer'])
DfCustomerUnique.head()

,Customer
0,21604822
1,21605122
2,21605922
3,21606122
4,21606522


In [272]:
#DfCustomerUnique

In [273]:
from scipy import sparse
from pandas.api.types import CategoricalDtype

rows = DataGrouped.Customer.astype(CategoricalDtype(categories=customers)).cat.codes # We have got 36 unique customers, which make up 13837 data rows (index)

# Get the associated row indices
cols = DataGrouped.SalesItem.astype(CategoricalDtype(categories= products)).cat.codes # We have got unique 3725 SalesItems, making up 13837 data rows (index)

# Get the associated column indices
#Compressed Sparse Row matrix
PurchaseSparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products))) #len of customers=35, len of products=3725
#csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
#where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]. , see https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

PurchaseSparse
#a sparse matrix is not a pandas dataframe, but sparse matrices are efficient for row slicing and fast matrix vector products


<884x300 sparse matrix of type '<class 'numpy.float64'>'
	with 2393 stored elements in Compressed Sparse Row format>

In [274]:
#We have 35 customers with 3725 items. For these user/item interactions, 13837 of these items had a purchase. 
#In terms of sparsity of the matrix, that makes:
MatrixSize = PurchaseSparse.shape[0]*PurchaseSparse.shape[1] # 130375 possible interactions in the matrix (35 unique customers * 3725 unique SalesItems=130375)
PurchaseAmount = len(PurchaseSparse.nonzero()[0]) # 13837 SalesItems interacted with; 
sparsity = 100*(1 - (PurchaseAmount/MatrixSize))
sparsity


99.09766214177978

Since we will use Matrix Factorization for our collaborative filtering it should not be a problem that 89.3% of the interaction matrix is sparse. In plain English, 89,3% in our case means that only 10,7% of our customer-item interactions are already filled, meaning that most items have not been purchased by customers. It is said that collaborative filtering can even work well with even more sparse data. We can prove that it works when checking our decent recommendings in the end. Cosine Similarity is a good measure for sparse data, so we will stick to Cosine (instead of Pearson, Euclidean or Manhattan).

# Recommending

We have already talked about sparsity. However, we will start with a simple recommender first, before we come to more advanced techniques also using optimization for sparse matrices. However, we can normalize items by purchase frequency across all users, which is done in section 3.3. below.

In [275]:
#for every dataset we will add a 1 as purchased. That means, that this customer has purchased this item, no matter how many. We use this binary data for our recommending. Another approach would be to use the SalesAmount and 
#normalize it, in case you want to treat the Amount of SalesItems purchased as a kind of taste factor, meaning that someone who bought SalesItem x 100 times, while another Customer bought that same SalesItem x only 5 times does 
#not like it as much. I believe, that very often in Sales a binary approach makes more sense, but of course that depends on the data.
def create_DataBinary(DataGrouped):
  # DataPrep must be DataGrouped?!
    DataBinary = DataPrep.copy()
    DataBinary['PurchasedYes'] = 1 
    return DataBinary

DataBinary = create_DataBinary(DataGrouped)
DataBinary.head()



,SalesItem,SalesAmount,Customer,PurchasedYes
0,پدیلانتوس بزرگ سبز,1.0,21619622,1
1,پدیلانتوس بزرگ سبز,1.0,22560122,1
2,پدیلانتوس متوسط سبز,1.0,41333922,1
3,پدیلانتوس متوسط سبز,1.0,25652422,1
4,پدیلانتوس متوسط سبز,1.0,22560122,1


In [276]:
data2=DataBinary.drop(['SalesAmount'], axis=1)
data2.head()

,SalesItem,Customer,PurchasedYes
0,پدیلانتوس بزرگ سبز,21619622,1
1,پدیلانتوس بزرگ سبز,22560122,1
2,پدیلانتوس متوسط سبز,41333922,1
3,پدیلانتوس متوسط سبز,25652422,1
4,پدیلانتوس متوسط سبز,22560122,1


In [277]:
#for better convenience we add I for Item for every SalesItem. Otherwise we would only have customer and SalesItem Numbers, which can be a little bit puzzling.
#data2['SalesItem'] = 'I' + data2['SalesItem'].astype(str)
data2['SalesItem'] = data2['SalesItem'].astype(str)

In [278]:

#DfMatrix = pd.pivot_table(data,index=["Customer"], columns='SalesItem')
DfMatrix = pd.pivot_table(data2, values='PurchasedYes', index='Customer', columns='SalesItem')
DfMatrix.head()

SalesItem,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,آگلونما سیلور متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
Customer,,,,,,,,,,,,,,,,,,,,,
21604822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21605122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21605922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21606122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21606522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [279]:
#since we are only using 1 and 0, we do not need to think about normalization. But talk is cheap, let`s check to see that even if we would normalize, the result is the same, of course:
DfMatrix=DfMatrix.fillna(0) #NaN values need to get replaced by 0, meaning they have not been purchased yet.
DfMatrixNorm3 = (DfMatrix-DfMatrix.min())/(DfMatrix.max()-DfMatrix.min())
DfMatrixNorm3.head()
#the proof is in the pudding. But we will come back to normalization later on again, when we will take real Sales Amount into consideration for recommending as well.

SalesItem,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,آگلونما سیلور متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
Customer,,,,,,,,,,,,,,,,,,,,,
21604822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21605122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21605922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21606122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21606522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [280]:
#we need to bring our pivot table into the desired format, via reset_index and rename_axis. 
DfResetted = DfMatrix.reset_index().rename_axis(None, axis=1) 
DfResetted.head()
#Now each row represents one customer`s buying behaviour: 1 means the customer has purchased, NaN the customer has not yet purchased it

,Customer,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
0,21604822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21605122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21605922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21606122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21606522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [281]:

DfMatrix.shape

(884, 300)

In [282]:
df=DfResetted #now working: because Customer must be nvarchar! If customer is int, then failure during CustItemSimilarity!

In [283]:
#we need to replace the NaN values with a 0, because our function will not work on NaN values.
#Please note, that we are only checking if a specific customer bought a specific item, yes or no. That is called binary. If customer bought a specific item, that means 1. If not, then 0. Because of this binary problem there is 
#no use in using any further scaling techniques.
df=df.fillna(0)
df.head()

,Customer,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
0,21604822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21605122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21605922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21606122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21606522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [284]:
#Creating a dataframe which only includes Sales Items. Customer is indexed instead.
DfSalesItem = df.drop('Customer', 1) 
DfSalesItem.head()

,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,آگلونما سیلور متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [285]:
#Calculate the Item based recommendation
import numpy as np
# We will normalize dataframe now, due to ..
#I believe we do not need to normalize, but let us compare..
#vectorized
DfSalesItemNorm = DfSalesItem / np.sqrt(np.square(DfSalesItem).sum(axis=0)) 
DfSalesItemNorm.head()

,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,آگلونما سیلور متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.301511,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [286]:
np.sum(DfSalesItem.iloc[:,0])

5.0

In [287]:
np.square(DfSalesItem).sum(axis=0)

آئونیوم متوسط سبز         5.0
آدینیوم کوچک             29.0
آرالیا متوسط سبز          1.0
آمستر سبز کوچک            1.0
آنتریوم قرمز متوسط        3.0
                         ... 
یوکا تنه دار سبز کوچک     3.0
یوکا سبز متوسط            2.0
یوکا سبز کوچک             4.0
یوکا متوسط ابلق           3.0
یوکا کوچک ابلق            1.0
Length: 300, dtype: float64

In [288]:
np.sqrt(np.square(DfSalesItem).sum(axis=0)) 

آئونیوم متوسط سبز        2.236068
آدینیوم کوچک             5.385165
آرالیا متوسط سبز         1.000000
آمستر سبز کوچک           1.000000
آنتریوم قرمز متوسط       1.732051
                           ...   
یوکا تنه دار سبز کوچک    1.732051
یوکا سبز متوسط           1.414214
یوکا سبز کوچک            2.000000
یوکا متوسط ابلق          1.732051
یوکا کوچک ابلق           1.000000
Length: 300, dtype: float64

In [289]:
# Calculating with Vectors to compute Cosine Similarities
ItemItemSim = DfSalesItemNorm.transpose().dot(DfSalesItemNorm) 
ItemItemSim.head()

,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,آگلونما سیلور متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
آئونیوم متوسط سبز,1.000000,0.166091,0.000000,0.0,0.0,0.0,0.100000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
آدینیوم کوچک,0.166091,1.000000,0.185695,0.0,0.0,0.0,0.083045,0.083045,0.0,0.0,...,0.0,0.061898,0.131306,0.0,0.0,0.0,0.131306,0.0,0.107211,0.0
آرالیا متوسط سبز,0.000000,0.185695,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
آمستر سبز کوچک,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
آنتریوم قرمز متوسط,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


In [290]:
'''#Another approach to the above would be using corr fct
#Where is the difference?
SalesItemCorrelation = DfSalesItem.corr()
SalesItemCorrelation.head()'''

'#Another approach to the above would be using corr fct\n#Where is the difference?\nSalesItemCorrelation = DfSalesItem.corr()\nSalesItemCorrelation.head()'

In [291]:
#ItemItemSim.to_excel("ExportItem-Item.xlsx")
# Create a placeholder items for closes neighbours to an item
ItemNeighbours = pd.DataFrame(index=ItemItemSim.columns,columns=range(1,10))
ItemNeighbours.head()

,1,2,3,4,5,6,7,8,9
آئونیوم متوسط سبز,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
آدینیوم کوچک,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
آرالیا متوسط سبز,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
آمستر سبز کوچک,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
آنتریوم قرمز متوسط,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [292]:
# Create a placeholder items for closes neighbours to an item
#ItemNeighbours = pd.DataFrame(index=ItemItemSim.columns,columns=range(1,10)) 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(ItemItemSim.columns)):
    ItemNeighbours.iloc[i,:9] = ItemItemSim.iloc[0:,i].sort_values(ascending=False)[:9].index
    #we only have 9 items, so we can max recommend 9 items (itself included)
 


In [293]:
ItemNeighbours.head()


,1,2,3,4,5,6,7,8,9
آئونیوم متوسط سبز,آئونیوم متوسط سبز,گلدان پلاستیکی قرمز f 141,گلدان فلزی طلایی متوسط استوانه ای پایه دار,گلدان پلاستیکی سفید f 141,ناز خرفه ای متوسط,گل قاشقی گلد,گلدان پلاستیکی آویز زرد گلپونه 5100,رویال سبز متوسط,برگ بیدی سبز کاسه ای
آدینیوم کوچک,آدینیوم کوچک,ساکولنت متوسط سبز,گلدان پلیمری مکعبی سفید 104-7.5,پیمانه پلاستیکی گلپونه قرمز,کاکتوس,گلدان فلزی طلایی متوسط استوانه ای پایه دار,جعبه کادویی مستطیلی مشکی کوچک,گلدان پلاستیکی قرمز f 141,گلدان پلیمری طرح دایره سفید 101-7.5
آرالیا متوسط سبز,آرالیا متوسط سبز,گل چمن عروس سبز,پیمانه پلاستیکی گلپونه قرمز,پایه چوبی سایز 17.5,زیر گلدانی پلاستیکی سفید 312,گل شفلرا متوسط سبز,آدینیوم کوچک,کراسولا درختی,خاک برگ کیسه سبز حجم 5 لیتر
آمستر سبز کوچک,آمستر سبز کوچک,گل شفلرا سوپر درختی سبز,آئونیوم متوسط سبز,گل قاشقی گلد,گل چمن عروس سبز,گل پلوما سبز,گل مصنوعی,گل محبوب شب بزرگ,گل مارانتا سبز متوسط
آنتریوم قرمز متوسط,آنتریوم قرمز متوسط,گلدان پلیمری مکعبی مشکی 104-20,برگ انجیری درختی بزرگ,فیکوس بلک بزرگ,برگ انجیری درختی متوسط,گلدان پلیمری مکعبی سفید 104-20,گلدان پلیمری استوانه ای سفید 103-25,پایه چوبی بلند سایز 20,پایه چوبی سایز 17.5


In [294]:
ItemNeighbours.head().iloc[:11,1:9]
#it needs to start at position 1, because position 0 is itself

,2,3,4,5,6,7,8,9
آئونیوم متوسط سبز,گلدان پلاستیکی قرمز f 141,گلدان فلزی طلایی متوسط استوانه ای پایه دار,گلدان پلاستیکی سفید f 141,ناز خرفه ای متوسط,گل قاشقی گلد,گلدان پلاستیکی آویز زرد گلپونه 5100,رویال سبز متوسط,برگ بیدی سبز کاسه ای
آدینیوم کوچک,ساکولنت متوسط سبز,گلدان پلیمری مکعبی سفید 104-7.5,پیمانه پلاستیکی گلپونه قرمز,کاکتوس,گلدان فلزی طلایی متوسط استوانه ای پایه دار,جعبه کادویی مستطیلی مشکی کوچک,گلدان پلاستیکی قرمز f 141,گلدان پلیمری طرح دایره سفید 101-7.5
آرالیا متوسط سبز,گل چمن عروس سبز,پیمانه پلاستیکی گلپونه قرمز,پایه چوبی سایز 17.5,زیر گلدانی پلاستیکی سفید 312,گل شفلرا متوسط سبز,آدینیوم کوچک,کراسولا درختی,خاک برگ کیسه سبز حجم 5 لیتر
آمستر سبز کوچک,گل شفلرا سوپر درختی سبز,آئونیوم متوسط سبز,گل قاشقی گلد,گل چمن عروس سبز,گل پلوما سبز,گل مصنوعی,گل محبوب شب بزرگ,گل مارانتا سبز متوسط
آنتریوم قرمز متوسط,گلدان پلیمری مکعبی مشکی 104-20,برگ انجیری درختی بزرگ,فیکوس بلک بزرگ,برگ انجیری درختی متوسط,گلدان پلیمری مکعبی سفید 104-20,گلدان پلیمری استوانه ای سفید 103-25,پایه چوبی بلند سایز 20,پایه چوبی سایز 17.5


In [295]:
ItemNeighbours.to_excel("1049738ExportItem-Item-data_neighbours.xlsx")

Now we will create a customer based recommendation which we need our item similarity matrix for. Then we will have a look which items our customers have bought and get the top N neighbours for each item. Afterwards we calculate the purchase history of the customer for each neighbour and calculate a similarity score for them. So in the end we just have to recommend the items with the highest score. 

In [296]:
#Now we will build a Customer based recommendation, which is build upon the item-item similarity matrix, which we have just calculated above.
# Create a place holder matrix for similarities, and fill in the customer column
CustItemSimilarity = pd.DataFrame(index=df.index,columns=df.columns)
CustItemSimilarity.iloc[:,:1] = df.iloc[:,:1]

In [297]:
CustItemSimilarity.head()

,Customer,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
0,21604822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21605122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21605922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21606122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21606522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [298]:
# Getting the similarity scores
def getScore(history, similarities):
   return sum(history*similarities)/(sum(similarities)+0.0001) 

In [299]:
# This takes ages (35 customers * 3725 items)
#We now loop through the rows and columns filling in empty spaces with similarity scores.
#Note that we score items that the customer has already consumed as 0, because there is no point recommending it again.
from timeit import default_timer as timer #to see how long the computation will take
start = timer()


for i in range(0,len(CustItemSimilarity.index)):
    for j in range(1,len(CustItemSimilarity.columns)):
        user = CustItemSimilarity.index[i]
        product = CustItemSimilarity.columns[j]
 
        if df.loc[i][j] == 1:
            CustItemSimilarity.loc[i][j] = 0
        else:
            ItemTop = ItemNeighbours.loc[product][1:9] #
            #do not use order but sort_values in latest pandas
            ItemTopSimilarity = ItemItemSim.loc[product].sort_values(ascending=False)[1:9]
            #here we will use the item dataframe, which we generated during item-item matrix 
            CustomerPurchasings = DfSalesItem.loc[user,ItemTop]
 
            CustItemSimilarity.loc[i][j] = getScore(CustomerPurchasings,ItemTopSimilarity)

end = timer()

print('\nRuntime: %0.2fs' % (end - start))

#if there occurs a strange error  tz=getattr(series.dtype, 'tz', None) .. pandas index.. then this might be if you have used int
# as column headers instead of string


Runtime: 738.72s


In [300]:
CustomerPurchasings

کود 20-20-20 پودری 70 گرمی    0.0
گلدان سفالی 50 تا 99          0.0
گل قاشقی گلد                  0.0
گل چمن عروس سبز               0.0
گل پلوما سبز                  0.0
گل مصنوعی                     0.0
گل محبوب شب بزرگ              0.0
گل مارانتا سبز متوسط          0.0
Name: 883, dtype: float64

In [301]:
ItemTopSimilarity

کود 20-20-20 پودری 70 گرمی    0.447214
گلدان سفالی 50 تا 99          0.072932
گل قاشقی گلد                  0.000000
گل چمن عروس سبز               0.000000
گل پلوما سبز                  0.000000
گل مصنوعی                     0.000000
گل محبوب شب بزرگ              0.000000
گل مارانتا سبز متوسط          0.000000
Name: یوکا کوچک ابلق, dtype: float64

In [302]:
user,product

(883, 'یوکا کوچک ابلق')

In [303]:
df.head()

,Customer,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
0,21604822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21605122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21605922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21606122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21606522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [304]:
CustItemSimilarity.head()

,Customer,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
0,21604822,0.0,0.0,0.0,0.0,0.0,0.439138,0.124107,0.0,0.122783,...,0.0,0.0,0.0,0.0,0.15561,0.054588,0.0,0.080362,0.0,0.140188
1,21605122,0.0,0.0,0.0,0.0,0.0,0.442058,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21605922,0.0,0.0,0.0,0.0,0.107149,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21606122,0.0,0.0,0.0,0.0,0.094496,0.096721,0.0,0.0,0.215038,...,0.0,0.0,0.0,0.0,0.341748,0.196036,0.0,0.080362,0.0,0.140188
4,21606522,0.0,0.0,0.04497,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [305]:
 ItemTopSimilarity

کود 20-20-20 پودری 70 گرمی    0.447214
گلدان سفالی 50 تا 99          0.072932
گل قاشقی گلد                  0.000000
گل چمن عروس سبز               0.000000
گل پلوما سبز                  0.000000
گل مصنوعی                     0.000000
گل محبوب شب بزرگ              0.000000
گل مارانتا سبز متوسط          0.000000
Name: یوکا کوچک ابلق, dtype: float64

In [306]:
ItemTop = ItemNeighbours.loc[product][1:9]

In [307]:
ItemItemSim.head()

,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,آگلونما سیلور متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
آئونیوم متوسط سبز,1.000000,0.166091,0.000000,0.0,0.0,0.0,0.100000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
آدینیوم کوچک,0.166091,1.000000,0.185695,0.0,0.0,0.0,0.083045,0.083045,0.0,0.0,...,0.0,0.061898,0.131306,0.0,0.0,0.0,0.131306,0.0,0.107211,0.0
آرالیا متوسط سبز,0.000000,0.185695,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
آمستر سبز کوچک,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
آنتریوم قرمز متوسط,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


In [308]:
CustItemSimilarity.head()

,Customer,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
0,21604822,0.0,0.0,0.0,0.0,0.0,0.439138,0.124107,0.0,0.122783,...,0.0,0.0,0.0,0.0,0.15561,0.054588,0.0,0.080362,0.0,0.140188
1,21605122,0.0,0.0,0.0,0.0,0.0,0.442058,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21605922,0.0,0.0,0.0,0.0,0.107149,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21606122,0.0,0.0,0.0,0.0,0.094496,0.096721,0.0,0.0,0.215038,...,0.0,0.0,0.0,0.0,0.341748,0.196036,0.0,0.080362,0.0,0.140188
4,21606522,0.0,0.0,0.04497,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [309]:
#now generate a matrix of customer based recommendations
CustItemRecommend = pd.DataFrame(index=CustItemSimilarity.index, columns=['Customer','item1','item2','item3','item4','item5','item6']) #Top 1,2..6
CustItemRecommend.head()

,Customer,item1,item2,item3,item4,item5,item6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [310]:
CustItemRecommend.iloc[0:,0] = CustItemSimilarity.iloc[:,0]
CustItemRecommend.head()

,Customer,item1,item2,item3,item4,item5,item6
0,21604822,NaN,NaN,NaN,NaN,NaN,NaN
1,21605122,NaN,NaN,NaN,NaN,NaN,NaN
2,21605922,NaN,NaN,NaN,NaN,NaN,NaN
3,21606122,NaN,NaN,NaN,NaN,NaN,NaN
4,21606522,NaN,NaN,NaN,NaN,NaN,NaN


In [311]:
#Instead of having the matrix filled with similarity scores we want to see the product names.
for i in range(0,len(CustItemSimilarity.index)):
    CustItemRecommend.iloc[i,1:] = CustItemSimilarity.iloc[i,:].sort_values(ascending=False).iloc[1:7,].index.transpose()

In [312]:
CustItemRecommend.head(20)


,Customer,item1,item2,item3,item4,item5,item6
0,21604822,بنجامین متوسط سبز,آگاو لاله ای سبز بزرگ,گل کالانکوا انگوری متوسط,گلدان پلیمری طرح دایره سفید 101-10,گل شفلرا نیم درختی بزرگ سبز,گلدان سفالی کمتر از 50
1,21605122,آگاو لاله ای سبز بزرگ,گلدان سفالی بیشتر از 100,فیکوس بلک بزرگ,گل قاشقی گلد,گل چمن عروس سبز,گل پلوما سبز
2,21605922,گلدان استوانه سفالی بیشتر از 100,گلدان پلاستیکی آبی f 144,گلدان پلاستیکی سرخابی f 145,پاندانوس ابلق متوسط شمشیری,ساکولنت بزرگ سبز,گلدان پلاستیکی صورتی f 142
3,21606122,گلدان پلاستیکی زرد f 145,بنجامین متوسط سبز,گلدان پلاستیکی قهوه ای f 164,رویال ارغوانی متوسط,گلدان پلیمری طرح دایره سفید 101-10,گل کالانکوا انگوری متوسط
4,21606522,گلدان پلاستیکی طوسی f 113,سیکاس سوزنی سوپر,کود 20-20-20 پودری 100 گرمی,رز هلندی زرد متوسط باغچه ای,خاک مکمل پیمانه ای گرین سی,گلدان پلیمری استوانه ای سفید 103-35
5,21607622,جعبه کادویی مستطیلی قهوه ای کوچک,یوکا تنه دار سبز سوپر,گلدان پلاستیکی سبز f 145,گل شفلرا نیم درختی بزرگ سبز,بنجامین متوسط سبز,یوکا تنه دار سبز متوسط
6,21608422,گلدان پلاستیکی طوسی f 114,سیکاس سوزنی سوپر,گلدان پلاستیکی طوسی f 113,کود 20-20-20 پودری 100 گرمی,رز هلندی زرد متوسط باغچه ای,خاک مکمل پیمانه ای گرین سی
7,21608722,گلدان پلاستیکی طوسی f 146,یوکا تنه دار سبز متوسط,فیکوس بلک بزرگ,گلدان پلیمری استوانه ای طوسی 103-17.5,گلدان فلزی طلایی متوسط استوانه ای پایه دار,گلدان پلاستیکی آویز زرد گلپونه 5100
8,21609022,جعبه کادویی مستطیلی قهوه ای کوچک,سانسوریا مینیاتوری گلد کوچک,سانسوریا مینیاتوری سبز کوچک,گلدان پلاستیکی آویزی سفید ZA 513,گلدان سفالی 50 تا 99,گل قاشقی سبز
9,21609222,گلدان فلزی آبی 45,گلدان پلیمری مکعبی طوسی 104-20,سانسوریا کراواتی مهتابی,گلدان گل پونه دایره ای f341,گل اختر هویا,سانسوریا ابلق شمشیری بلند


In [313]:
filter=df['Customer']==21607122


In [314]:
#PurchaseSparse.nonzero()[0]
#df.toarray().nonzero()[0]

In [315]:
print(df.where(filter))

     Customer  آئونیوم متوسط سبز   آدینیوم کوچک  آرالیا متوسط سبز   \
0         NaN                 NaN           NaN                NaN   
1         NaN                 NaN           NaN                NaN   
2         NaN                 NaN           NaN                NaN   
3         NaN                 NaN           NaN                NaN   
4         NaN                 NaN           NaN                NaN   
..        ...                 ...           ...                ...   
879       NaN                 NaN           NaN                NaN   
880       NaN                 NaN           NaN                NaN   
881       NaN                 NaN           NaN                NaN   
882       NaN                 NaN           NaN                NaN   
883       NaN                 NaN           NaN                NaN   

     آمستر سبز کوچک  آنتریوم قرمز متوسط  آگاو لاله ای سبز بزرگ  \
0               NaN                 NaN                    NaN   
1               NaN        

In [316]:
df

,Customer,آئونیوم متوسط سبز,آدینیوم کوچک,آرالیا متوسط سبز,آمستر سبز کوچک,آنتریوم قرمز متوسط,آگاو لاله ای سبز بزرگ,آگلونما برفی بزرگ,آگلونما برفی متوسط,آگلونما بلک متوسط,...,گیاه مرجان قرمز,یوکا تنه دار ابلق بزرگ,یوکا تنه دار سبز بزرگ,یوکا تنه دار سبز سوپر,یوکا تنه دار سبز متوسط,یوکا تنه دار سبز کوچک,یوکا سبز متوسط,یوکا سبز کوچک,یوکا متوسط ابلق,یوکا کوچک ابلق
0,21604822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21605122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21605922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21606122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21606522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,41803622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
880,41803822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
881,41803922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
882,41804022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [317]:

CustItemRecommend.to_excel("1049738ExportCustomer-Item-CustItemRecommend.xlsx")
#We have coded a binary recommender engine, which works only sufficient on a small data set. Let us see in the next chapter if we can enhance the performance and scalability.

In [318]:
#CustItemRecommend.pivot(columns='Customer')
#for i in range():


In [319]:
'''from sklearn.neighbors import NearestNeighbors
knn= NearestNeighbers(metric='cosine',algorithm='brute',n_neighbors=20,n_jobs=-1)'''


"from sklearn.neighbors import NearestNeighbors\nknn= NearestNeighbers(metric='cosine',algorithm='brute',n_neighbors=20,n_jobs=-1)"

In [320]:
'''#conn = pyodbc.connect('Driver={SQL Server};Server=192.168.100.17\sql2019;Database=Modern_Master;uid=sa;pwd=PAYA+master')
cursor = conn.cursor()
cursor.execute("DELETE FROM bi.PrsClusters ;")
conn.commit()
                  
for i in range(0,len(cluster_map)):
           
             
             
    cursor.execute("INSERT INTO bi.PrsClusters (IdPrs,ClusterIndex)  VALUES (?,?);"  ,k1[i],k2[i])'''
#conn.commit()